In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter


###########################
# Data Manipulation / Model
###########################
# username, password, and CRUD Python module name
username = "aacuser"
password = "801312"
shelter = AnimalShelter(username, password)

# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash("SimpleExample")

# add Grazioso Salvare's logo to dashboard
image_filename = "Grazioso-Salvare-Logo.png" # image file
encoded_image = base64.b64encode(open(image_filename, "rb").read())

app.layout = html.Div([
    html.Div(id="hidden-div", style={"display":"none"}),
    html.Div(html.Img(src="data:image/png;base64,{}".format(encoded_image.decode()),style={"height": "10%", "width": "10%"}),style={"textAlign": "center"}),
    html.Div(html.Center(html.B(html.H2("Jonathan Sussan: Project Two Dashboard")))),
    html.Center(html.P("Welcome, please make a selection...")),
    html.Hr(),
    html.Div(
        # radio buttons for the interactive filtering options
        dcc.RadioItems(
            id="filter-type",
            options=[
            {"label": "Water Rescue", "value": "water_rescue"},
            {"label": "Mountain or Wilderness Rescue", "value": "mountain_rescue"},
            {"label": "Disaster or Individual Tracking", "value": "disaster_tracking"},
            {"label": "Default Widgets", "value": "reset"}
            ],
            value="reset",
            labelStyle={"display": "inline-block"}
        )
    ),
    html.Hr(),
    dt.DataTable(
        id="datatable-id",
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict("records"),
        # features for interactive data table to make it user-friendly
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="multi",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
    # sets up the dashboard so that the chart and geolocation chart are side-by-side
    html.Div(className="row",
         style={"display" : "flex"},
             children=[
        html.Div(
            id="graph-id",
            className="col s12 m6",

            ),
        html.Div(
            id="map-id",
            className="col s12 m6",
            )
        ])
])


#############################################
# Interaction Between Components / Controller
#############################################

@app.callback([Output("datatable-id","data"),
               Output("datatable-id","columns")],
              [Input("filter-type", "value")])
# code to filter interactive data table with MongoDB queries
def update_dashboard(filter_type):
        # filter for water rescue selection
        if filter_type == "water_rescue":
            df = pd.DataFrame(list(shelter.read({"$and": [{"sex_upon_outcome": "Intact Female"},
                                                          {"$or": [
                                                              {"breed": "Chesapeake Bay Retriever"},
                                                              {"breed": "Labrador Retriever Mix"},
                                                              {"breed": "Newfoundland"}]
                                                          },
                                                          {"$and": [{"age_upon_outcome_in_weeks": {"$gte": 26}},
                                                                    {"age_upon_outcome_in_weeks": {"$lte": 156}}]
                                                          }]
                                                })))
        # filter for mountain rescue selection
        elif filter_type == "mountain_rescue":
            df = pd.DataFrame(list(shelter.read({"$and": [{"sex_upon_outcome": "Intact Male"},
                                                          {"$or": [
                                                              {"breed": "Alaskan Malamute"},
                                                              {"breed": "German Shepherd"},
                                                              {"breed": "Old English Sheepdog"},
                                                              {"breed": "Rottweiler"},
                                                              {"breed": "Siberian Husky"}]
                                                          },
                                                          {"$and": [{"age_upon_outcome_in_weeks": {"$gte": 26}},
                                                                    {"age_upon_outcome_in_weeks": {"$lte": 156}}]
                                                          }]
                                                })))
        # filter for disaster or individual tracking selection
        elif filter_type == "disaster_tracking":
            df = pd.DataFrame(list(shelter.read({"$and": [{"sex_upon_outcome": "Intact Male"},
                                                          {"$or": [
                                                              {"breed": "Bloodhound"},
                                                              {"breed": "Doberman Pinscher"},
                                                              {"breed": "German Shepherd"},
                                                              {"breed": "Golden Retriever"},
                                                              {"breed": "Rottweiler"}]
                                                          },
                                                          {"$and": [{"age_upon_outcome_in_weeks": {"$gte": 20}},
                                                                    {"age_upon_outcome_in_weeks": {"$lte": 300}}]
                                                          }]
                                                })))
        # returns all widgets to their original, unfiltered state
        elif filter_type == "reset":
            df = pd.DataFrame.from_records(shelter.read({}))
        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict("records")        
        return (data,columns)

@app.callback(
    Output("datatable-id", "style_data_conditional"),
    [Input("datatable-id", "selected_columns")]
)
def update_styles(selected_columns):
    return [{
        "if": { "column_id": i },
        "background_color": "#D2F3FF"
    } for i in selected_columns]

@app.callback(
    Output("graph-id", "children"),
    [Input("datatable-id", "derived_viewport_data")])

def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    names = dff["breed"].value_counts().keys().tolist()
    values = dff["breed"].value_counts().tolist()
    # code for the pie chart
    return [
        dcc.Graph(            
            figure = px.pie(
                data_frame=dff, 
                values = values, 
                names = names,
                # color pie chart with plotly color combination Jet 
                color_discrete_sequence=px.colors.sequential.Jet,
                width=800, 
                height=500
            )
        )
    ]

@app.callback(
    Output("map-id", "children"),
    [Input("datatable-id", "derived_viewport_data"),
     Input("datatable-id", "selected_rows"),
     Input("datatable-id", "selected_columns")])

def update_map(viewData, selected_rows, selected_columns):
    # code for the geolocation chart, shows current selection
    dff = pd.DataFrame.from_dict(viewData)
    selected_rows = [0]
    # creates a map for a single selected row or the default
    if len(selected_rows) == 1:
        return [
            dl.Map(style={"width":"800px", "height": "500px"}, center=[30.75,-97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
            
                # marker with tool tip and popup
                dl.Marker(position=[(dff.iloc[selected_rows[0],13]), (dff.iloc[selected_rows[0],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[0],4]),
                    dl.Popup([
                        html.H4("Animal Name"),
                        html.P(dff.iloc[selected_rows[0],9])
                    ])
                ])
            ])
        ]



app